<a href="https://colab.research.google.com/github/massivetexts/llm_aut_study/blob/main/notebooks/PrepareSemDisForEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prepare code for scoring in http://semdis.wlu.psu.edu/, then process the output.

In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
base_dir = Path('drive/MyDrive/Grants/MOTES/') #@param { type: 'raw' }
gt_dir = base_dir / 'Data' / 'aut_ground_truth' #@param { type: 'raw' }
evaldir = base_dir / 'Data' / 'evaluation' #@param { type: 'raw' }
print("GT options", [x.name.split('.')[0] for x in gt_dir.glob('*tar.gz')])
split = 'gt_main2' #@param ['gt_byparticipant', 'gt_main2', 'gt_byprompt'] {allow-input: true}

# copy locally and unzip
!cp "{gt_dir}/{split}.tar.gz" .
!rm -rf data
!tar -xf {split}.tar.gz
data_dir = Path(f"data/{split}")
print("Data decompressed to", data_dir)

GT options ['gt_main', 'gt_bypart3', 'gt_byprompt4', 'gt_byparticipant', 'gt_byprompt', 'all', 'gt_main2', 'gt_main_std']
Data decompressed to data/gt_main2


In [ ]:
testdata_df = pd.DataFrame([pd.read_json(x, orient='index')[0] for x in (data_dir / 'test').glob('*json')])
testdata_df.prompt = testdata_df.prompt.str.replace('paperclip', 'paper clip') # improve prompt
testdata_df.shape

(3030, 9)

In [ ]:
x = testdata_df[['id', 'prompt', 'response']]
x.columns = ['id', 'item', 'response']
parts = 3
for i in range(parts):
    x.iloc[i::parts].to_csv(f'{split}-{i}.csv') # save multiple files

## Process data for analysis format

In [ ]:
!mkdir -p {evaldir}/{split}
scored = pd.concat([pd.read_csv(p) for p in Path('.').glob('SemD*csv')])
scoring_cols = [col for col in scored.columns if col.startswith('SemDis') and col != 'SemDis_factor']

for metric in scoring_cols:
    # don't join on response because semdis adjusts the text sometimes
    out = scored[['id', 'item', metric]].copy()
    out.columns = ['id', 'prompt', 'predicted']
    out['split'] = split
    out['model'] = metric
    #scored.iloc[:,5:]
    out = testdata_df.merge(out, how='left', on=['id', 'prompt'])
    out.prompt = out.prompt.str.replace('paper clip', 'paperclip')
    print('NAs', out.predicted.isna().sum())
    out[['id', 'model', 'participant', 'prompt', 'target', 'predicted', 'src',
        'split', 'question', 'response', 'response_num']].to_csv(evaldir / split / f'{metric}.csv')
    out.predicted.isna().sum()

NAs 20
NAs 18
NAs 18
NAs 29
NAs 19
NAs 14


In [ ]:
print(evaldir / split / f'{metric}.csv')
out.groupby(['prompt'])[['target', 'predicted']].corr()['target'].loc[(slice(None), 'predicted')]

drive/MyDrive/Grants/MOTES/Data/evaluation/gt_main2/SemDis_MEAN.csv


prompt
backpack      0.088868
ball          0.087153
book          0.221470
bottle        0.102755
box           0.105734
brick         0.109117
fork          0.190020
hat           0.300573
knife         0.013340
lightbulb     0.275774
pants         0.339713
paperclip     0.134830
pencil        0.329377
rope          0.187856
shoe          0.237699
shovel        0.097057
sock          0.040200
spoon         0.233097
table         0.335739
tire          0.356340
toothbrush    0.164354
Name: target, dtype: float64